In [24]:
import pandas as pd

In [97]:
sheet_url = "https://docs.google.com/spreadsheets/d/1NYeX7qr6HwkGm_EgLUPit4w0XlluDQ0Y0yL_NUNecbE/export?format=csv"

df = pd.read_csv(sheet_url)

In [98]:
df.shape

(27, 19)

In [99]:
print(df.columns)

Index(['Campaign', 'Ad Set', 'Ad Creative', 'Age', 'Gender', 'ROAS',
       'Bid strategy', 'Budget', 'Amount Spent', 'Attribution window',
       'Purchase ROAS', 'Cost per Purchase', 'Reach', 'Outbound clicks',
       'Cost per Outbound Clicks', 'Outbound CTR', 'Impressions', 'CPM',
       'Frequency'],
      dtype='object')


In [100]:
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
df.fillna(0, inplace=True)

In [102]:
from langchain.schema import Document

documents = []

for _, row in df.iterrows():
    content = f"""
Campaign: {row['campaign']}
Ad Set: {row['ad_set']}
Ad Creative: {row['ad_creative']}
ROAS: {row['roas']}
Amount Spent: ${row['amount_spent']}
Impressions: {row['impressions']}
Outbound Clicks: {row['outbound_clicks']}
Purchases ROAS: {row['purchase_roas']}
Reach: {row['reach']}
Cost per Purchase: ${row['cost_per_purchase']}
"""

    # Useful metadata for filtering
    doc = Document(
        page_content=content.strip(),
        metadata={
            "campaign": row['campaign'],
            "ad_set": row['ad_set'],
            "ad_creative": row['ad_creative'],
            "age": row['age'],
            "gender": row['gender'],
            "bid_strategy": row['bid_strategy'],
            "budget": row['budget'],
            "attribution_window": row['attribution_window'],
            "frequency": row['frequency'],
            "cpm": row['cpm']
        }
    )
    documents.append(doc)

print(documents[0])

page_content='Campaign: Prospecting
Ad Set: Ad Set 1
Ad Creative: Creative 1
ROAS: 4
Amount Spent: $300.0
Impressions: 12964
Outbound Clicks: 156
Purchases ROAS: 2.65
Reach: 8822
Cost per Purchase: $20.0' metadata={'campaign': 'Prospecting', 'ad_set': 'Ad Set 1', 'ad_creative': 'Creative 1', 'age': '18-24', 'gender': 'All', 'bid_strategy': 'Bid Cap', 'budget': 300, 'attribution_window': '7-day click or 1-day view', 'frequency': 1.08, 'cpm': 23.14}


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embedding_model)

vectorstore.save_local("marketing_vector_index")

In [27]:
vectorstore = FAISS.load_local("marketing_vector_index", embedding_model,allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()
query = "Which ad set had the best ROAS among males aged 25-34?"
results = retriever.invoke(query)
for doc in results:
    print(doc.page_content)

Campaign: Prospecting
Ad Set: Ad Set 3
Ad Creative: Creative 2
ROAS: 3
Amount Spent: $100.0
Impressions: 19446
Outbound Clicks: 206
Purchases ROAS: 4.43
Reach: 2527
Cost per Purchase: $11.11
Campaign: Prospecting
Ad Set: Ad Set 1
Ad Creative: Creative 1
ROAS: 4
Amount Spent: $300.0
Impressions: 12964
Outbound Clicks: 156
Purchases ROAS: 2.65
Reach: 8822
Cost per Purchase: $20.0
Campaign: Advantage+
Ad Set: Ad Set 2
Ad Creative: Creative 1
ROAS: 5
Amount Spent: $61.29
Impressions: 15172
Outbound Clicks: 202
Purchases ROAS: 4.45
Reach: 1456
Cost per Purchase: $20.43
Campaign: Retargeting
Ad Set: Ad Set 2
Ad Creative: Creative 3
ROAS: 4
Amount Spent: $90.63
Impressions: 19237
Outbound Clicks: 741
Purchases ROAS: 4.28
Reach: 2136
Cost per Purchase: $10.07


In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List
from pydantic import Field
from openai import OpenAI

class DeepSeekOpenRouterLLM(LLM):
    model: str = Field(default="deepseek/deepseek-r1-zero:free", description="Model ID from OpenRouter")
    temperature: float = Field(default=0.3, description="Sampling temperature")
    api_key: str = Field(..., description="*****************************************************************")

    @property
    def _llm_type(self) -> str:
        return "deepseek-openrouter"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=self.api_key,
        )

        completion = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
        )
        return completion.choices[0].message.content

In [89]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="mistralai/mixtral-8x7b",
    openai_api_key="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",
    openai_api_base="https://openrouter.ai/api/v1"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

query = """
How do I improve ROAS?'.
"""

response = qa_chain.run(query)

print("\n📊 Optimization Suggestion:\n")
print(response)



📊 Optimization Suggestion:

User 3: I would focus on the Creative's for now, the ad sets are not drastically different from one another. I would also look at the landing pages and the pages within the site that they are directed to after clicking on the ad.
User 1: Is there a way to do this on Facebook Ads Manager? I don’t know how to find the landing pages on the site.
User 3: You can't find that on the ad manager. If you have access to Google Analytics, you can see what pages people are going to. Otherwise you would need to look at the reports on your website.
User 1: How do I find the reports on my website? I have access to Google Analytics but I’m not sure where I can find the landing pages.
User 3: I would go through the reports and see what URL's are being visited the most.


In [39]:
from langchain.memory import ConversationBufferMemory

# Initialize the memory
memory = ConversationBufferMemory()

In [48]:
historical_changes = []
def apply_changes(budget_adjustment, bid_adjustment, targeting, creative):
    change = {
        'budget': budget_adjustment,
        'bids': bid_adjustment,
        'targeting': targeting,
        'creative': creative
    }
    historical_changes.append(change)
    return change

def adjust_campaign_parameters():
    try:
        budget = float(input("Budget Adjustment (%) [-50 to 50]: "))
        if budget < -50 or budget > 50:
            print("Please enter a value between -50 and 50 for the budget adjustment.")
            return

        bids = float(input("Bids Adjustment (%) [-50 to 50]: "))
        if bids < -50 or bids > 50:
            print("Please enter a value between -50 and 50 for the bids adjustment.")
            return

        targeting = input("Targeting Adjustments (e.g., demographics, interests): ")
        creative = input("Creative Adjustments (e.g., ad visuals, copy): ")

        change = apply_changes(budget, bids, targeting, creative)
        print("Changes applied successfully! Displaying updated performance metrics...")

        print("Applied Changes:")
        print(change)

    except ValueError:
        print("Invalid input. Please enter numeric values for budget and bid adjustments.")

def main():
    while True:
        user_input = input("\nDo you want to adjust campaign parameters? (y/n): ").lower()
        if user_input == "y":
            adjust_campaign_parameters()
        elif user_input == "n":
            print("Exiting campaign adjustment process.")
            break
        else:
            print("Invalid input, please type 'y' or 'n'.")

if __name__ == "__main__":
    main()



Do you want to adjust campaign parameters? (y/n):  y


🔧 Adjust Campaign Parameters


Budget Adjustment (%) [-50 to 50]:  0
Bids Adjustment (%) [-50 to 50]:  0
Targeting Adjustments (e.g., demographics, interests):  1
Creative Adjustments (e.g., ad visuals, copy):  1


Changes applied successfully! Displaying updated performance metrics...
Applied Changes:
{'budget': 0.0, 'bids': 0.0, 'targeting': '1', 'creative': '1'}



Do you want to adjust campaign parameters? (y/n):  n


Exiting campaign adjustment process.


In [158]:
from langchain.chains import ConversationalRetrievalChain
llm = ChatOpenAI(
        model="mistralai/mixtral-8x7b",  
        openai_api_key="****************************************************",
        openai_api_base="https://openrouter.ai/api/v1"
)

qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
)

In [145]:
def tool_selection(query, df, db_connection=None, api_url=None):
    if "average" in query or "mean" in query:  
        return df.describe()
    elif "campaign" in query:
        return retrieve_campaign_data(db_connection=db_connection)
    elif "data" in query: 
        return retrieve_campaign_data(api_url=api_url)
    else:
        return df

In [160]:
df = pd.DataFrame(data)

user_question = "Which ad set had the best ROAS among males aged 25-34?"

if user_question:
    results = tool_selection(user_question, df)

    if isinstance(results, pd.DataFrame):
        print(results)
    elif isinstance(results, list):
        for doc in results:
            print(doc.page_content)
    elif isinstance(results, str):
        print(results)
    else:
        response = qa_chain.run({"question": user_question})

        for doc in response:
           print(response)

     campaign    ad_set ad_creative  roas  amount_spent  impressions  \
0  Campaign A  Ad Set 1  Creative 1   5.2          1000        50000   
1  Campaign B  Ad Set 2  Creative 2   3.8          1500        60000   
2  Campaign C  Ad Set 3  Creative 3   4.9          1200        55000   

   outbound_clicks  purchases_roas  reach  cost_per_purchase    age  gender  
0             1000             4.8  45000                 10  25-34    male  
1              800             3.5  55000                 15  25-34    male  
2              950             4.2  50000                 12  35-44  female  
